<a href="https://colab.research.google.com/github/vsangireddy27/BellFootball/blob/main/whatsnextimproved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from collections import defaultdict

def combine_concepts(df):
    df['Concept'] = df['PASS ROUTE CONCEPT'].combine_first(df['RUN CONCEPT'])
    return df

def calculate_scenario_frequencies(df_scenario):
    formation_after_list = []
    play_after_list = []
    concept_after_list = []
    for i in range(len(df_scenario) - 1):
        next_row = df_scenario.iloc[i + 1]
        formation_after_list.append(next_row['OFF FORM'])
        play_after_list.append(next_row['OFF PLAY'])
        concept_after_list.append(next_row['Concept'])
    results_df = pd.DataFrame({
        'Formation_After': formation_after_list,
        'Play_After': play_after_list,
        'Concept_After': concept_after_list
    })
    formation_counts = results_df.groupby(['Formation_After', 'Play_After']).size().reset_index(name='Formation_Frequency')
    play_counts = results_df.groupby(['Play_After', 'Formation_After']).size().reset_index(name='Play_Frequency')
    concept_counts = results_df.groupby(['Concept_After', 'Formation_After', 'Play_After']).size().reset_index(name='Concept_Frequency')
    return formation_counts, play_counts, concept_counts

def analyze_datasets(file_paths):
    aggregated_formations = defaultdict(int)
    aggregated_plays = defaultdict(int)
    aggregated_concepts = defaultdict(int)
    concept_names = defaultdict(str)
    scenario_totals = defaultdict(int)
    for file_path in file_paths:
        df = pd.read_excel(file_path)
        df = combine_concepts(df)
        df_filtered = df[df['ODK'] == 'O'].reset_index(drop=True)
        run_scenarios = {
            "Run < 0": df_filtered[(df_filtered['PLAY TYPE'] == 'Run') & (df_filtered['GN/LS'] < 0)],
            "Run = 0": df_filtered[(df_filtered['PLAY TYPE'] == 'Run') & (df_filtered['GN/LS'] == 0)],
            "Run < 5": df_filtered[(df_filtered['PLAY TYPE'] == 'Run') & (df_filtered['GN/LS'] < 5)],
            "Run >= 5": df_filtered[(df_filtered['PLAY TYPE'] == 'Run') & (df_filtered['GN/LS'] >= 5)],
            "Run > 10": df_filtered[(df_filtered['PLAY TYPE'] == 'Run') & (df_filtered['GN/LS'] > 10)],
        }
        pass_scenarios = {
            "Pass < 0": df_filtered[(df_filtered['PLAY TYPE'] == 'Pass') & (df_filtered['GN/LS'] < 0)],
            "Pass = 0": df_filtered[(df_filtered['PLAY TYPE'] == 'Pass') & (df_filtered['GN/LS'] == 0)],
            "Pass < 5": df_filtered[(df_filtered['PLAY TYPE'] == 'Pass') & (df_filtered['GN/LS'] < 5)],
            "Pass >= 5": df_filtered[(df_filtered['PLAY TYPE'] == 'Pass') & (df_filtered['GN/LS'] >= 5)],
            "Pass > 10": df_filtered[(df_filtered['PLAY TYPE'] == 'Pass') & (df_filtered['GN/LS'] > 10)],
            "Pass/Run > 20": df_filtered[(df_filtered['GN/LS'] > 20)],
        }
        scenarios = {**run_scenarios, **pass_scenarios}
        for scenario_name, scenario_df in scenarios.items():
            formation_counts, play_counts, concept_counts = calculate_scenario_frequencies(scenario_df)
            for _, row in formation_counts.iterrows():
                key = (scenario_name, row['Formation_After'], row['Play_After'])
                aggregated_formations[key] += row['Formation_Frequency']
            for _, row in play_counts.iterrows():
                key = (scenario_name, row['Play_After'], row['Formation_After'])
                aggregated_plays[key] += row['Play_Frequency']
            for _, row in concept_counts.iterrows():
                key = (scenario_name, row['Formation_After'], row['Play_After'])
                aggregated_concepts[key] += row['Concept_Frequency']
                concept_names[key] = row['Concept_After']
            scenario_totals[scenario_name] += len(scenario_df) - 1
    compiled_results = []
    for (scenario_name, formation, play), formation_count in aggregated_formations.items():
        play_count = aggregated_plays.get((scenario_name, play, formation), 0)
        concept_count = aggregated_concepts.get((scenario_name, formation, play), 0)
        concept_name = concept_names.get((scenario_name, formation, play), "")
        total = scenario_totals[scenario_name]
        formation_percentage = (formation_count / total) * 100 if total > 0 else 0
        play_percentage = (play_count / total) * 100 if total > 0 else 0
        concept_percentage = (concept_count / total) * 100 if total > 0 else 0
        compiled_results.append({
            "Scenario": scenario_name,
            "Formation": formation,
            "Formation_Frequency": formation_count,
            "Formation_Percentage": formation_percentage,
            "Formation_Ratio": f"{formation_count}/{total}",
            "Play": play,
            "Play_Frequency": play_count,
            "Play_Percentage": play_percentage,
            "Play_Ratio": f"{play_count}/{total}",
            "Concept_Name": concept_name,
            "Concept_Frequency": concept_count,
            "Concept_Percentage": concept_percentage,
            "Concept_Ratio": f"{concept_count}/{total}",
        })
    compiled_df = pd.DataFrame(compiled_results)
    compiled_df = compiled_df.sort_values(by="Formation_Frequency", ascending=False)
    return compiled_df
file_paths = [
    '/content/SHC vs Mitty.xlsx',
    '/content/SHC vs Riordan.xlsx',
    '/content/SHC vs SF.xlsx',
    '/content/SHC vs SI.xlsx',
    '/content/SHC vs Serra.xlsx',
    '/content/SHC vs VC.xlsx'
]

compiled_df = analyze_datasets(file_paths)
print("\nCompiled Results with Combined Concept Column:")
print(compiled_df)
print("\n" + "="*50 + "\n")
output_path = "/content/Book1.xlsx"
compiled_df.to_excel(output_path, index=False)



Compiled Results with Combined Concept Column:
      Scenario    Formation  Formation_Frequency  Formation_Percentage  \
37    Pass < 5  DOUBLE SLOT                   28             26.168224   
8    Pass >= 5  DOUBLE SLOT                   18             20.000000   
32    Pass = 0  DOUBLE SLOT                   18             29.032258   
9    Pass >= 5  DOUBLE SLOT                   13             14.444444   
58     Run < 5        TRIPS                   12             13.953488   
..         ...          ...                  ...                   ...   
68    Pass = 0        DEUCE                    1              1.612903   
65    Pass < 0        DEUCE                    1              5.882353   
64    Pass < 0        DEUCE                    1              5.882353   
63    Run > 10  DOUBLE SLOT                    1             33.333333   
143   Pass < 5        TRIPS                    1              0.934579   

    Formation_Ratio         Play  Play_Frequency  Play_Percenta